In [1]:
!pip install nba_api
!pip install sqlalchemy-redshift
!pip install redshift_connector
!pip install rsa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 5.8 MB/s eta 0:00:00


In [1]:
#Manejo de API
#https://rapidapi.com/theapiguy/api/free-nba/details
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playerawards

In [2]:
#Manipulación de AWS
#https://aws.amazon.com/blogs/big-data/use-the-amazon-redshift-sqlalchemy-dialect-to-interact-with-amazon-redshift/
import sqlalchemy as sa
from sqlalchemy import orm as sa_orm
from sqlalchemy.engine.url import URL

#Cifrado y carga de archivo de config
import rsa, json, base64, os
from os import path

import pandas as pa

In [3]:
#Variables y constantes globales
CONFIG_PATH = 'Archivos'

config_dic = { #Diccionario con la configuración de la app
    'request_timout': 50,   #Timeout para los requests
    'player_limit': 500,      #Límite de jugadores a procesar. -1: Sin limite.
    'error_limit': 10       #Límite de errores a partir del cual se aborta la carga.
} 

In [4]:
def cargar_configuracion():
    #Completa el diccionario global con la configuración necesaria

    #Setear directorio
    fullpath = path.join(os.getcwd(), CONFIG_PATH)
    os.chdir(fullpath)
    print(f'Directorio actual: {os.getcwd()}')

    #Cargar private key
    with open('private.pem', 'r') as file:
        privKey = rsa.PrivateKey.load_pkcs1(file.read())

    #Cargar archivo de configuración
    with open('config.json', 'r') as file:
        jConfig = json.load(file)

    config_dic['cHost'] = jConfig['host']
    config_dic['cPort'] = rsa.decrypt(base64.b64decode(jConfig['cPort']), privKey).decode()
    config_dic['cDatabase'] = rsa.decrypt(base64.b64decode(jConfig['cDatabase']), privKey).decode()
    config_dic['cUser'] = rsa.decrypt(base64.b64decode(jConfig['cUser']), privKey).decode()
    config_dic['cPass'] = rsa.decrypt(base64.b64decode(jConfig['cPass']), privKey).decode()

cargar_configuracion()

Directorio actual: /home/coder/working_dir/Archivos


In [5]:
##  *************** EXTRACT *************** 
print(f">>>> Comienza el proceso de extracción")

#Recuperar los equipos (guardar resultados de la API como DF)
teams_df = pa.DataFrame(teams.get_teams())
teams_df

>>>> Comienza el proceso de extracción


,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [8]:
#Corroborar tipos de datos inferidos
teams_df.dtypes

id               int64
full_name       object
abbreviation    object
nickname        object
city            object
state           object
year_founded     int64
dtype: object

In [7]:
#Recuperar los jugadores (guardar resultados de la API como DF)
players_df = pa.DataFrame(players.get_players())
players_df

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4810,1627790,Ante Zizic,Ante,Zizic,False
4811,78647,Jim Zoet,Jim,Zoet,False
4812,78648,Bill Zopf,Bill,Zopf,False
4813,1627826,Ivica Zubac,Ivica,Zubac,True


In [9]:
#Corroborar tipos de datos inferidos
players_df.dtypes

id             int64
full_name     object
first_name    object
last_name     object
is_active       bool
dtype: object

In [17]:
#Recuperar los premios desde el CSV (se armó utilizando la API, se leé desde CSV a fines prácticos pues a veces fallaba la API)
premios_df = pa.read_csv('premios.csv')
print(f"Jugadores resguardados en CSV: {len(premios_df['PERSON_ID'].drop_duplicates().to_list())}")
premios_df

Jugadores resguardados en CSV: 961


,PERSON_ID,FIRST_NAME,LAST_NAME,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3
0,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,2.0,1969-70,NaN,NaN,1610612749,Award,Kia Motors,KIADT,NaN
1,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,2.0,1970-71,NaN,NaN,1610612749,Award,Kia Motors,KIADT,NaN
2,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,1.0,1973-74,NaN,NaN,1610612749,Award,Kia Motors,KIADT,NaN
3,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,1.0,1974-75,NaN,NaN,1610612749,Award,Kia Motors,KIADT,NaN
4,76003,Kareem,Abdul-Jabbar,Los Angeles Lakers,All-Defensive Team,2.0,1975-76,NaN,NaN,1610612747,Award,Kia Motors,KIADT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,203092,Tyler,Zeller,Cleveland Cavaliers,All-Rookie Team,2,2012-13,NaN,NaN,1610612739,Award,Kia Motors,KIART,NaN
5230,1917,Wang,Zhizhi,China,Olympic Appearance,NaN,1996,NaN,NaN,NaN,Award,Olympic,Appearance,NaN
5231,1917,Wang,Zhizhi,China,Olympic Appearance,NaN,2000,NaN,NaN,NaN,Award,Olympic,Appearance,NaN
5232,1917,Wang,Zhizhi,China,Olympic Appearance,NaN,2008,NaN,NaN,NaN,Award,Olympic,Appearance,NaN


In [18]:
#Corroborar tipos de dato inferidos
premios_df.dtypes

PERSON_ID               int64
FIRST_NAME             object
LAST_NAME              object
TEAM                   object
DESCRIPTION            object
ALL_NBA_TEAM_NUMBER    object
SEASON                 object
MONTH                  object
WEEK                   object
CONFERENCE             object
TYPE                   object
SUBTYPE1               object
SUBTYPE2               object
SUBTYPE3               object
dtype: object

In [19]:
#ALL_NBA_TEAM_NUMBER debe ser un INT
premios_df['ALL_NBA_TEAM_NUMBER'] = premios_df['ALL_NBA_TEAM_NUMBER'].str.replace('.0', '') #Eliminar formato de float
premios_df['ALL_NBA_TEAM_NUMBER'] = premios_df['ALL_NBA_TEAM_NUMBER'].str.replace('(null)', '0') #Eliminar valores que no son número
premios_df['ALL_NBA_TEAM_NUMBER'].fillna('0',inplace=True) #Completar los campos en NULL con 0
premios_df['ALL_NBA_TEAM_NUMBER'] = premios_df['ALL_NBA_TEAM_NUMBER'].astype('int32')
#MONTH y WEEK deben ser date
premios_df['MONTH'] = premios_df['MONTH'].astype('datetime64[ns]')
premios_df['WEEK'] = premios_df['WEEK'].astype('datetime64[ns]')
premios_df.dtypes

PERSON_ID                       int64
FIRST_NAME                     object
LAST_NAME                      object
TEAM                           object
DESCRIPTION                    object
ALL_NBA_TEAM_NUMBER             int32
SEASON                         object
MONTH                  datetime64[ns]
WEEK                   datetime64[ns]
CONFERENCE                     object
TYPE                           object
SUBTYPE1                       object
SUBTYPE2                       object
SUBTYPE3                       object
dtype: object

In [22]:
#Explorar el DF
#premios_df.head()
#premios_df.tail()
premios_df

,PERSON_ID,FIRST_NAME,LAST_NAME,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3
0,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,2,1969-70,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN
1,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,2,1970-71,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN
2,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,1,1973-74,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN
3,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,1,1974-75,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN
4,76003,Kareem,Abdul-Jabbar,Los Angeles Lakers,All-Defensive Team,2,1975-76,NaT,NaT,1610612747,Award,Kia Motors,KIADT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,203092,Tyler,Zeller,Cleveland Cavaliers,All-Rookie Team,2,2012-13,NaT,NaT,1610612739,Award,Kia Motors,KIART,NaN
5230,1917,Wang,Zhizhi,China,Olympic Appearance,0,1996,NaT,NaT,NaN,Award,Olympic,Appearance,NaN
5231,1917,Wang,Zhizhi,China,Olympic Appearance,0,2000,NaT,NaT,NaN,Award,Olympic,Appearance,NaN
5232,1917,Wang,Zhizhi,China,Olympic Appearance,0,2008,NaT,NaT,NaN,Award,Olympic,Appearance,NaN


In [33]:
#Revisión de nulos
def has_null_columns(dataframe):
    return len(dataframe.columns[dataframe.isna().all()].tolist()) != 0

print("Revisión de columnas vacías...")
print(f"DF Teams: {has_null_columns(teams_df)}")
print(f"DF Players: {has_null_columns(players_df)}")
print(f"DF Premios: {has_null_columns(premios_df)}")

Revisión de columnas vacías...
DF Teams: False
DF Players: False
DF Premios: False


In [36]:
#Revisión de Duplicados
def has_duplicates (dataframe):
    return len(premios_df[premios_df.duplicated()]) != 0

print("Control de duplicados..")
if has_duplicates(teams_df):
    print("DF Teams... Tiene duplicados")
    teams_df.drop_duplicates(inplace=True)
else:
    print("DF Teams.... OK")

if has_duplicates(players_df):
    print("DF Players... Tiene duplicados")
    players_df.drop_duplicates(inplace=True)
else:
    print("DF Players.... OK")
    
if has_duplicates(premios_df):
    print("DF Premios... Tiene duplicados")
    premios_df.drop_duplicates(inplace=True)
else:
    print("DF Premios.... OK")

Control de duplicados..
DF Teams.... OK
DF Players.... OK
DF Premios.... OK


In [44]:
def df_memory_usage(dataframe):
    return round(dataframe.memory_usage(deep=True).sum()/(1024*1024),2) #Retorna valor en MB

print("Chequeo de uso de memoria...")

print(f"DF Teams: {df_memory_usage(teams_df)} mb")
print(f"DF Players: {df_memory_usage(players_df)} mb")
print(f"DF Premios: {df_memory_usage(premios_df)} mb")

Chequeo de uso de memoria...
DF Teams: 0.01 mb
DF Players: 0.94 mb
DF Premios: 3.2 mb


In [49]:
print("Análisis preliminar de la data...")
print("\n")
print(f"Cantidad de equipos: {len(teams_df)}")
print(f"Cantidad de jugadores: {len(players_df)}")
print(f"Cantidad de premios: {len(premios_df)}")
print("\n")
print(f"Cantidad de premios x Jugador: \n {premios_df[['PERSON_ID','FIRST_NAME','LAST_NAME']].value_counts()}")

Análisis preliminar de la data...


Cantidad de equipos: 30
Cantidad de jugadores: 4815
Cantidad de premios: 5234


Cantidad de premios x Jugador: 
 PERSON_ID  FIRST_NAME  LAST_NAME  
2544       LeBron      James          154
893        Michael     Jordan          94
977        Kobe        Bryant          86
1495       Tim         Duncan          75
406        Shaquille   O'Neal          68
                                     ... 
76531      Mike        Davis            1
101178     Arvydas     Macijauskas      1
77328      Tom         LaGarde          1
101162     Marcin      Gortat           1
76918      Chuck       Halbert          1
Name: count, Length: 1003, dtype: int64


In [69]:
##  *************** TRANSFORM *************** 
"""El objetivo es ajustar algunos datos con los DF y finalmente hacer un merge para poder hacer la posterior carga en Redshift."""

#Ajustar players_df: Se agrega columna PERSON_ID para poder hacer el join con premios_df
players_df['PERSON_ID'] = players_df['id']

#Merge de premios_df con players_df
factica_df = premios_df[['PERSON_ID','TEAM','DESCRIPTION','ALL_NBA_TEAM_NUMBER','SEASON','MONTH','WEEK','CONFERENCE','TYPE','SUBTYPE1','SUBTYPE2','SUBTYPE3']].merge(players_df[['first_name','last_name','is_active','PERSON_ID']], on='PERSON_ID', how='left')

#Renombrar columnas de jugadores
factica_df['PLAYER_FST_NAME'] = factica_df['first_name']
factica_df['PLAYER_LST_NAME'] = factica_df['last_name']
factica_df['PLAYER_IS_ACTIVE'] = factica_df['is_active']

#Eliminar columnas redundantes
factica_df.drop(['first_name','last_name','is_active'], axis=1, errors='ignore', inplace=True)
factica_df

,PERSON_ID,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3,PLAYER_FST_NAME,PLAYER_LST_NAME,PLAYER_IS_ACTIVE
0,76003,Milwaukee Bucks,All-Defensive Team,2,1969-70,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,Kareem,Abdul-Jabbar,False
1,76003,Milwaukee Bucks,All-Defensive Team,2,1970-71,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,Kareem,Abdul-Jabbar,False
2,76003,Milwaukee Bucks,All-Defensive Team,1,1973-74,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,Kareem,Abdul-Jabbar,False
3,76003,Milwaukee Bucks,All-Defensive Team,1,1974-75,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,Kareem,Abdul-Jabbar,False
4,76003,Los Angeles Lakers,All-Defensive Team,2,1975-76,NaT,NaT,1610612747,Award,Kia Motors,KIADT,NaN,Kareem,Abdul-Jabbar,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,203092,Cleveland Cavaliers,All-Rookie Team,2,2012-13,NaT,NaT,1610612739,Award,Kia Motors,KIART,NaN,Tyler,Zeller,False
5230,1917,China,Olympic Appearance,0,1996,NaT,NaT,NaN,Award,Olympic,Appearance,NaN,Wang,Zhi-zhi,False
5231,1917,China,Olympic Appearance,0,2000,NaT,NaT,NaN,Award,Olympic,Appearance,NaN,Wang,Zhi-zhi,False
5232,1917,China,Olympic Appearance,0,2008,NaT,NaT,NaN,Award,Olympic,Appearance,NaN,Wang,Zhi-zhi,False


In [70]:
#Ajustar players_df: Se agrega columna TEAM_ID y se convierte a mayusculas FULL_NAME para poder hacer el join con premios_df utilizando el atributo TEAM
teams_df['TEAM'] = teams_df['full_name'].str.upper() #Agregar el nombre del equipo en mayúsculas como TEAM
teams_df['TEAM_ID'] = teams_df['id'].astype('object') #Se agrega el team_id como object, para evitar problemas de conversión
teams_df

#Ajustar factica_df: Se convierte a mayuscula el atributo TEAM para poder hacer el join
factica_df['TEAM'] = factica_df['TEAM'].str.upper()

#Merge de premios_df con teams_df
factica_df = factica_df[['PERSON_ID','PLAYER_FST_NAME','PLAYER_LST_NAME','PLAYER_IS_ACTIVE','TEAM','DESCRIPTION','ALL_NBA_TEAM_NUMBER','SEASON','MONTH','WEEK','CONFERENCE','TYPE','SUBTYPE1','SUBTYPE2','SUBTYPE3']].merge(teams_df[['TEAM_ID','abbreviation','city','state','year_founded','TEAM']], on='TEAM', how='left')

#En la fáctica se corrigen atributos asociados al TEAM
factica_df['TEAM_ABBREVIATION'] = factica_df['abbreviation']
factica_df['TEAM_CITY'] = factica_df['city']
factica_df['TEAM_STATE'] = factica_df['state']
factica_df['TEAM_YEAR_FOUNDED'] = factica_df['year_founded']
factica_df.drop(['abbreviation','city','state','year_founded'], axis=1, errors='ignore', inplace=True) #Se eliminan los atributos anteriores

#Corregir el tipo de dato del TEAM_ID y tambien TEAM_YEAR_FOUNDED
"""Primero debo eliminar valores nulos"""
factica_df['TEAM_ID'].fillna('0',inplace=True)
factica_df['TEAM_YEAR_FOUNDED'].fillna('0',inplace=True)
"""Finalmente los casteo"""
factica_df['TEAM_ID'] = factica_df['TEAM_ID'].astype('int64')
factica_df['TEAM_YEAR_FOUNDED'] = factica_df['TEAM_YEAR_FOUNDED'].astype('int32')
factica_df

,PERSON_ID,PLAYER_FST_NAME,PLAYER_LST_NAME,PLAYER_IS_ACTIVE,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,TEAM_STATE,TEAM_YEAR_FOUNDED
0,76003,Kareem,Abdul-Jabbar,False,MILWAUKEE BUCKS,All-Defensive Team,2,1969-70,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,1610612749,MIL,Milwaukee,Wisconsin,1968
1,76003,Kareem,Abdul-Jabbar,False,MILWAUKEE BUCKS,All-Defensive Team,2,1970-71,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,1610612749,MIL,Milwaukee,Wisconsin,1968
2,76003,Kareem,Abdul-Jabbar,False,MILWAUKEE BUCKS,All-Defensive Team,1,1973-74,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,1610612749,MIL,Milwaukee,Wisconsin,1968
3,76003,Kareem,Abdul-Jabbar,False,MILWAUKEE BUCKS,All-Defensive Team,1,1974-75,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,1610612749,MIL,Milwaukee,Wisconsin,1968
4,76003,Kareem,Abdul-Jabbar,False,LOS ANGELES LAKERS,All-Defensive Team,2,1975-76,NaT,NaT,1610612747,Award,Kia Motors,KIADT,NaN,1610612747,LAL,Los Angeles,California,1948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,203092,Tyler,Zeller,False,CLEVELAND CAVALIERS,All-Rookie Team,2,2012-13,NaT,NaT,1610612739,Award,Kia Motors,KIART,NaN,1610612739,CLE,Cleveland,Ohio,1970
5230,1917,Wang,Zhi-zhi,False,CHINA,Olympic Appearance,0,1996,NaT,NaT,NaN,Award,Olympic,Appearance,NaN,0,NaN,NaN,NaN,0
5231,1917,Wang,Zhi-zhi,False,CHINA,Olympic Appearance,0,2000,NaT,NaT,NaN,Award,Olympic,Appearance,NaN,0,NaN,NaN,NaN,0
5232,1917,Wang,Zhi-zhi,False,CHINA,Olympic Appearance,0,2008,NaT,NaT,NaN,Award,Olympic,Appearance,NaN,0,NaN,NaN,NaN,0


In [71]:
#Chequear tipos de datos del DF de la factica
factica_df.dtypes

PERSON_ID                       int64
PLAYER_FST_NAME                object
PLAYER_LST_NAME                object
PLAYER_IS_ACTIVE                 bool
TEAM                           object
DESCRIPTION                    object
ALL_NBA_TEAM_NUMBER             int32
SEASON                         object
MONTH                  datetime64[ns]
WEEK                   datetime64[ns]
CONFERENCE                     object
TYPE                           object
SUBTYPE1                       object
SUBTYPE2                       object
SUBTYPE3                       object
TEAM_ID                         int64
TEAM_ABBREVIATION              object
TEAM_CITY                      object
TEAM_STATE                     object
TEAM_YEAR_FOUNDED               int32
dtype: object

In [72]:
#Limpieza de datos del DF de la fáctica
print(f"Cantidad de registros originales: {len(factica_df)}")

"""Se eliminan premios que NO tienen equipo asociado"""
factica_df = factica_df[factica_df['TEAM_ID']!=0]

"""Se eliminan duplicados"""
if has_duplicates(players_df):
    players_df.drop_duplicates(inplace=True)
    
print(f"Cantidad de registros finales: {len(factica_df)}")
factica_df

Cantidad de registros originales: 5234
Cantidad de registros finales: 4180


,PERSON_ID,PLAYER_FST_NAME,PLAYER_LST_NAME,PLAYER_IS_ACTIVE,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,TEAM_STATE,TEAM_YEAR_FOUNDED
0,76003,Kareem,Abdul-Jabbar,False,MILWAUKEE BUCKS,All-Defensive Team,2,1969-70,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,1610612749,MIL,Milwaukee,Wisconsin,1968
1,76003,Kareem,Abdul-Jabbar,False,MILWAUKEE BUCKS,All-Defensive Team,2,1970-71,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,1610612749,MIL,Milwaukee,Wisconsin,1968
2,76003,Kareem,Abdul-Jabbar,False,MILWAUKEE BUCKS,All-Defensive Team,1,1973-74,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,1610612749,MIL,Milwaukee,Wisconsin,1968
3,76003,Kareem,Abdul-Jabbar,False,MILWAUKEE BUCKS,All-Defensive Team,1,1974-75,NaT,NaT,1610612749,Award,Kia Motors,KIADT,NaN,1610612749,MIL,Milwaukee,Wisconsin,1968
4,76003,Kareem,Abdul-Jabbar,False,LOS ANGELES LAKERS,All-Defensive Team,2,1975-76,NaT,NaT,1610612747,Award,Kia Motors,KIADT,NaN,1610612747,LAL,Los Angeles,California,1948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5220,1629027,Trae,Young,True,ATLANTA HAWKS,NBA Player of the Week,0,2021-22,NaT,2021-11-22,East,Award,Kia Motors,KIPWK,NaN,1610612737,ATL,Atlanta,Georgia,1949
5221,1629027,Trae,Young,True,ATLANTA HAWKS,NBA Player of the Week,0,2021-22,NaT,2022-01-17,East,Award,Kia Motors,KIPWK,NaN,1610612737,ATL,Atlanta,Georgia,1949
5222,1629027,Trae,Young,True,ATLANTA HAWKS,NBA Player of the Week,0,2021-22,NaT,2022-03-28,East,Award,Kia Motors,KIPWK,NaN,1610612737,ATL,Atlanta,Georgia,1949
5223,1629027,Trae,Young,True,ATLANTA HAWKS,All-NBA,3,2021-22,NaT,NaT,East,Award,Kia Motors,KIANT,NaN,1610612737,ATL,Atlanta,Georgia,1949


In [75]:
##  *************** LOAD ***************
"""Establecer la conexión con la BD y cargar la información contenida en el DF de la fáctica."""

def redshift_conectar():
    #Crear string de conexión y generar el engine

    rs_url = URL.create(
        drivername='redshift+redshift_connector',
        host= config_dic['cHost'],
        port= config_dic['cPort'],
        database= config_dic['cDatabase'],
        username= config_dic['cUser'],
        password= config_dic['cPass']
    )
    return sa.create_engine(rs_url)

def redshift_crear_factica(table_name, engine):
    #Crea la tabla fáctica en RedShift
    metadata = sa.MetaData()

    factTable = sa.Table(
        table_name, metadata,
        sa.Column('id', sa.BIGINT),                     #fact table index
        sa.Column('player_id', sa.BIGINT),              #player.id
        sa.Column('player_fst_name', sa.VARCHAR(100)),  #player.first_name
        sa.Column('player_lst_name', sa.VARCHAR(100)),  #player.last_name
        sa.Column('player_is_active', sa.BOOLEAN),      #player.is_active
        sa.Column('team_id', sa.BIGINT),                #teams.id
        sa.Column('team', sa.VARCHAR(256)),             #teams.full_name
        sa.Column('team_abb', sa.VARCHAR(10)),          #teams.abbreviation
        sa.Column('team_city', sa.VARCHAR(100)),        #teams.city
        sa.Column('team_state', sa.VARCHAR(100)),       #teams.state
        sa.Column('team_year_founded', sa.INT),         #teams.year_founded
        sa.Column('description', sa.VARCHAR(256)),      #playerawards.description
        sa.Column('all_nba_team_number', sa.INT),       #playerawards.all_nba_team_number
        sa.Column('season', sa.VARCHAR(10)),            #playerawards.season
        sa.Column('award_type', sa.VARCHAR(25)),        #playerawards.type
        sa.Column('award_subtype1', sa.VARCHAR(25)),    #playerawards.subtype1
        sa.Column('award_subtype2', sa.VARCHAR(25)),    #playerawards.subtype2
        redshift_diststyle='KEY',
        redshift_distkey='season',
        redshift_sortkey='player_id')

    #Si existe la tabla, la elimino
    if sa.inspect(engine).has_table(table_name):
        factTable.drop(bind=engine)
    # Crear la tabla
    factTable.create(bind=engine)

    return factTable

def checkInt(value):
#Se utiliza para verificar que el valor obtenido sea INT, caso contrario retorna 0.
    try:
        return int(value)
    except:
        return 0

In [111]:
#Conectar a Redshift y crear la sesion
redshift_engine = redshift_conectar()

Session = sa_orm.sessionmaker()
Session.configure(bind=redshift_engine)

In [112]:
#Crear la tabla fáctica de los premios por jugador por temporada por equipo
table_name = 'f_premios_obtenidos'

rsPremiosFactTable = redshift_crear_factica(table_name, redshift_engine)

In [ ]:
print(f">>>> Comienza la carga de la fáctica {table_name}")

idxPremio = 0   #Indice de la fáctica
errorCount = 0  #Cantidad de errores en el proceso

factica_df.reset_index()

with Session() as session:
    for idx in factica_df.index:
        try:
            #print(f"Procesando: {idx}")
            
            idxPremio += 1  #Aumenta el indice de la fáctica

            rsPremiosFactTable.insert().values()
            insert_data_row = rsPremiosFactTable.insert().values(
                id = idxPremio,
                player_id = checkInt(factica_df['PERSON_ID'][idx]),
                player_fst_name = factica_df['PLAYER_FST_NAME'][idx],
                player_lst_name = factica_df['PLAYER_LST_NAME'][idx],
                player_is_active = factica_df['PLAYER_IS_ACTIVE'][idx],
                team_id = checkInt(factica_df['TEAM_ID'][idx]),
                team = factica_df['TEAM'][idx],
                team_abb = factica_df['TEAM_ABBREVIATION'][idx],
                team_city = factica_df['TEAM_CITY'][idx],
                team_state = factica_df['TEAM_STATE'][idx],
                team_year_founded = checkInt(factica_df['TEAM_YEAR_FOUNDED'][idx]),
                description = factica_df['DESCRIPTION'][idx],
                all_nba_team_number = checkInt(factica_df['ALL_NBA_TEAM_NUMBER'][idx]),
                season = factica_df['SEASON'][idx],
                award_type = factica_df['TYPE'][idx],
                award_subtype1 = factica_df['SUBTYPE1'][idx],
                award_subtype2 = factica_df['SUBTYPE2'][idx]
            )
            
            session.execute(insert_data_row)   #Probar: bulk_save_objects(lst_of_rows)
            session.commit()
            
        except Exception as e:
            print(f"Error al insertar registro de id {idxPremio}. Msj: {e}")
            errorCount += 1
            continue
            
print(f">>>> Finaliza la carga de la fáctica {table_name}")         

>>>> Comienza la carga de la fáctica f_premios_obtenidos


In [ ]:
#Obtengo la cantidad de filas en la fáctica
with Session() as session:
    nFactRows = session.query(rsPremiosFactTable).count()

print(f"Premios insertados en la fáctica: {nFactRows}")
print(f"Cantidad de errores durante el proceso: {errorCount}")